# <span style="color:teal">RBFE Network - Analysis</span>


In [9]:
# import libraries
import BioSimSpace as BSS
import os
import glob
import csv
import numpy as np
import math
import pandas as pd
import networkx as nx
import yaml
from scipy.stats import sem as sem
from scipy.stats import bootstrap
from sklearn.metrics import mean_absolute_error as mae
import pickle
import tempfile
import itertools
import matplotlib.pyplot as plt
import csv
import numpy as np
import pandas as pd 
from rdkit import Chem

# import from path for freenrgworkflows.
import sys
sys.path.insert(1, '/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis/') # FIXME this will need to be adjusted in the future
import networkanalysis
import experiments
import stats

# try w cinnabar
sys.path.insert(1, '/home/anna/Documents/cinnabar')
import cinnabar
from cinnabar import wrangle,plotting
print(cinnabar.__file__)

try:
    import pipeline
except:
    print("adding code to the pythonpath...")
    code = '/home/anna/Documents/code/python'
    if code not in sys.path:
        sys.path.insert(1, code)
    import pipeline

from pipeline import *
from pipeline.utils import validate
from pipeline.analysis import *

/home/anna/Documents/cinnabar/cinnabar/__init__.py


In [2]:
# folders
protein = "tyk2"
file_ext = "MBAR_alchemlyb_benchmark" # for results files

# define all the folder locations
bench_folder = f"/home/anna/Documents/benchmark"
main_folder = f"{bench_folder}/{protein}_benchmark"
out_folder = f"{main_folder}/outputs"
res_folder = f"../test/results"
temp_folder = f"{main_folder}/outputs/results/temp"
exp_folder = f"{bench_folder}/inputs/experimental"

# make folders that may not exist
folder_list = [res_folder, temp_folder]
for fold in folder_list:
    validate.folder_path(fold, create=True)

# files
net_file = f"{main_folder}/execution_model/network_lomap.dat"
exp_file = f"{exp_folder}/{protein}.yml"
exp_file_dat = f"{res_folder}/exp_data_{protein}.dat"

# OUTPUT
file_ext_out = "test" # for how files will be written

# files that will get written
comp_pert_file_name = f"computed_perturbations_average_{file_ext_out}"
cinnabar_file = f"cinnabar_format_{file_ext_out}"
# TODO add file names for saving graphs

In [3]:
# we will define the engine here for ease
engines = ["SOMD","AMBER","GROMACS"]

results_dict = {}

for eng in engines:
    eng_object = analysis(
        glob.glob(f"{out_folder}/repeat_*_{eng}_{file_ext}.csv"),
        exp_file,
        net_file
        )
    results_dict[eng] = eng_object


In [5]:
# can generate graph to visualise
# for eng in engines:
#     graph_obj = graph(values_dict[eng]["ligs"],values_dict[eng]["perts"])
#     graph_obj.draw_graph(file_dir = f"{res_folder}")

In [6]:
# using freenergworkflows to convert
# # first need to convert the yml file into one useable by freenergworkflows
convert.yml_into_freenrgworkflows(exp_file, exp_file_dat)
experiments = experiments.ExperimentalData()
experiments.compute_affinities(exp_file_dat, data_type="IC50", comments="#", delimiter=",")
experimental_DDGs = experiments.freeEnergiesInKcal
exp_pert_dict,exp_lig_dict = make_dict.experimental_from_freenrgworkflows(experimental_DDGs, all_ligands, all_perturbations)
values_dict["experimental"]["perts"] = all_perturbations
values_dict["experimental"]["ligs"] = all_ligands
values_dict["experimental"]["pert_results"] = exp_pert_dict
values_dict["experimental"]["val_results"] = exp_lig_dict
values_dict["experimental"]["freenrgworkflows_ouput"] = experimental_DDGs


In [7]:
# do without freenrgworkflows
exper_val_dict = convert.yml_into_exper_dict(exp_file, exp_file_dat)
exp_pert_dict,exp_lig_dict = make_dict.experimental_from_cinnabar(exper_val_dict, all_ligands, all_perturbations)
values_dict["experimental"]["perts"] = all_perturbations
values_dict["experimental"]["ligs"] = all_ligands
values_dict["experimental"]["pert_results"] = exp_pert_dict
values_dict["experimental"]["val_results"] = exp_lig_dict

In [ ]:
# plotting


In [ ]:
# cycle closures
for eng in engines:

    pert_dict = values_dict[eng]["pert_results"]
    cycle_closures = # get from the network graph object

    cycles = pipeline.analysis.make_dict_cycle_closures(pert_dict, cycle_closures)    

    print(f"{eng} cycle vals is {cycles[1]}")
    print(f"{eng} cycle mean is {cycles[2]}")
    print(f"{eng} cycle deviation is {cycles[3]}")

    values_dict[eng]["cycle_closures"] = (cycles[0]) # the cycles dict

In [ ]:
from cinnabar import wrangle,plotting

In [ ]:
for eng in engines:
    convert_cinnabar_file(values_dict[eng]["results_files"], values_dict["experimental"]["val_results"], f"{res_folder}/{cinnabar_file}_{eng}")

    network = wrangle.FEMap(f"{res_folder}/{cinnabar_file}_{eng}.csv")
    # plot the perturbations
    plotting.plot_DDGs(network.graph, filename=f"{res_folder}/DDGs_{file_ext_out}_{eng}.png", title=f"DDGs_{protein}_{file_ext_out}_{eng}")
    #plot the ligands
    plotting.plot_DGs(network.graph, filename=f"{res_folder}/DGs_{file_ext_out}_{eng}.png", title=f"DGs_{protein}_{file_ext_out}_{eng}")